In [27]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class GraphAttentionLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat

        self.W = nn.Parameter(torch.empty(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.empty(size=(2*out_features, 1))) # 尝试："1"应该换为一个超参数t
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, h, adj):
        Wh = torch.mm(h, self.W) # h.shape:(N, in_features), Wh.shape:(N, out_features)
        e = self._prepare_attentional_mechanism_input(Wh)
        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj>0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime = torch.matmul(attention, Wh)
        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

    def _prepare_attentional_mechanism_input(self, Wh):
        # Wh.shape (N, out_feature)
        # self.a.shape (2 * out_feature, 1)
        # Wh1&2.shape (N, 1)
        # e.shape (N, N)

        Wh1 = torch.matmul(Wh, self.a[:self.out_features, :])
        Wh2 = torch.matmul(Wh, self.a[self.out_features:, :])
        # a[:self.out_features, :]用于被处理的节点i
        # a[self.out_features:, :]用于节点i邻域内的节点j
        e = Wh1 * Wh2.T #broadcast add 注：源代码用的是"+" 尝试：对比二者效果
        # e为N个节点中任意两个节点之间的相关度组成的矩阵(N*N)
        return self.leakyrelu(e)

    def __repr__(self):
        return self.__class__.__name__+"("+str(self.in_features)+"->"+str(self.out_features)+")"

In [28]:
class GAT(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout, alpha, nheads):
        super(GAT, self).__init__()
        self.dropout = dropout
        self.attentions = [GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module("attention_{}".format(i), attention)
        self.out_att = GraphAttentionLayer(nhid * nheads, nclass, dropout=dropout, alpha=alpha, concat=False)
    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training = self.training)
        x = torch.cat([att(x, adj) for att in self.attentions], dim=1) # 每个attention的输出维度为8, 8个attention拼接即得64维
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.elu(self.out_att(x, adj)) # out_att的输入维度为64, 输出维度为7, 即种类数
        return F.log_softmax(x, dim=1)

In [29]:
import scipy.sparse as sp
def encode_onehot(labels):
    classes = sorted(list(set(labels)))
    classes_dict = {c:np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

def load_data(path="./cora/", dataset="cora"):
    print("Loading {} dataset...".format(dataset))
    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset), dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j:i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset), dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), shape=(labels.shape[0], labels.shape[0]), dtype=np.float32)
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    features = normalize_features(features)
    adj = normalize_adj(adj+sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)
    adj = torch.FloatTensor(np.array(adj.todense()))
    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    # print(idx_train.dtype)
    idx_train = torch.LongTensor(idx_train) # 此行与下面两行未注释的语句可省略
    # print(idx_train.dtype)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)
    return adj, features, labels, idx_train, idx_val, idx_test

def normalize_adj(mx):
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    # return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)
    mid = np.dot(r_mat_inv_sqrt, mx)
    return np.dot(mid, r_mat_inv_sqrt)

def normalize_features(mx):
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0
    r_mat_inv = sp.diags(r_inv)
    return mx

def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels).reshape(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [30]:
# # 理解.transpose()
# a = np.array([[1, 2, 3], [4, 5, 6]])
# print(a)
# print(a.transpose())
# # 理解自定义的accuracy函数
# b = np.array([[1, 2, 2], [4, 5, 5]])
# c = torch.tensor(a)
# d = torch.tensor(b)
# print(c.eq(d))
# print(c.eq(d).sum())

In [31]:
import os
import glob
import time
import random
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

parser = argparse.ArgumentParser()
parser.add_argument("--no_cuda", action="store_true", default=False, help="Disables CUDA training.")
parser.add_argument("--fastmode", action="store_true", default=False, help="Validate during training pass.")
parser.add_argument("--sparse", action="store_true", default=False, help="GAT with sparse version or not.")
parser.add_argument("--seed", type=int, default=72, help="Random seed.")
parser.add_argument("--epochs", type=int, default=100, help="Number of epochs to train.")
parser.add_argument("--lr", type=float, default=0.005, help="Initial learning rate.")
parser.add_argument("--weight_decay", type=float, default=5e-4, help="Weight dacay (L2 loss on parameters).")
parser.add_argument("--hidden", type=int, default=8, help="Number of hidden units.")
parser.add_argument("--nb_heads", type=int, default=8, help="Number of head attentions.")
parser.add_argument("--dropout", type=float, default=0.6, help="Dropout rate (1 - keep probability).")
parser.add_argument("--alpha", type=float, default=0.2, help="Alpha for the leaky_relu.")
parser.add_argument("--patience", type=int, default=100, help="Patience")

# args = parser.parse_args()
args =parser.parse_known_args()[0]
# print(args)
# print(parser)
args.cuda = not args.no_cuda and torch.cuda.is_available()
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

adj, features, labels, idx_train, idx_val, idx_test = load_data()
model = GAT(nfeat=features.shape[1], nhid=args.hidden, nclass=int(labels.max())+1, dropout=args.dropout, nheads=args.nb_heads, alpha=args.alpha)
optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

if args.cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

features, adj, labels = Variable(features), Variable(adj), Variable(labels)

def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()
    if not args.fastmode:
        model.eval()
        output = model(features, adj)
    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.data.item()),
          'acc_train: {:.4f}'.format(acc_train.data.item()),
          'loss_val: {:.4f}'.format(loss_val.data.item()),
          'acc_val: {:.4f}'.format(acc_val.data.item()),
          'time: {:.4f}s'.format(time.time() - t))
    return loss_val.data.item()

def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.data.item()),
          "accuracy= {:.4f}".format(acc_test.data.item()))

Loading cora dataset...


In [32]:
t_total = time.time()
loss_values = []
bad_counter = 0
best = args.epochs + 1
best_epoch = 0
for epoch in range(args.epochs):
    loss_values.append(train(epoch))
    torch.save(model.state_dict(), "{}.pkl".format(epoch))
    if loss_values[-1] < best:
        best = loss_values[-1]
        best_epoch = epoch
        bad_counter = 0
    else:
        bad_counter += 1
    if bad_counter == args.patience:
        break

    files = glob.glob("*.pkl")
    for file in files:
        epoch_nb = int(file.split(".")[0]) #
        if epoch_nb < best_epoch:
            os.remove(file)

files = glob.glob('*.pkl')
for file in files:
    epoch_nb = int(file.split('.')[0])
    if epoch_nb > best_epoch:
        os.remove(file)

print("Optimization Finished!")
print("Total time: {:.4f}s".format(time.time() - t_total))
print('Loading {}th epoch'.format(best_epoch))
model.load_state_dict(torch.load('{}.pkl'.format(best_epoch)))

Epoch: 0001 loss_train: 2.1246 acc_train: 0.0643 loss_val: 1.8932 acc_val: 0.2467 time: 4.5609s
Epoch: 0002 loss_train: 1.9298 acc_train: 0.2214 loss_val: 1.7696 acc_val: 0.4967 time: 4.6212s
Epoch: 0003 loss_train: 1.9016 acc_train: 0.2714 loss_val: 1.6614 acc_val: 0.5400 time: 4.6821s
Epoch: 0004 loss_train: 1.7966 acc_train: 0.4000 loss_val: 1.5664 acc_val: 0.5933 time: 4.6584s
Epoch: 0005 loss_train: 1.5643 acc_train: 0.4857 loss_val: 1.4833 acc_val: 0.6200 time: 4.6320s
Epoch: 0006 loss_train: 1.5821 acc_train: 0.4643 loss_val: 1.4089 acc_val: 0.6733 time: 4.6671s
Epoch: 0007 loss_train: 1.4673 acc_train: 0.4786 loss_val: 1.3407 acc_val: 0.6900 time: 4.6748s
Epoch: 0008 loss_train: 1.5597 acc_train: 0.5071 loss_val: 1.2787 acc_val: 0.7067 time: 4.6611s
Epoch: 0009 loss_train: 1.4175 acc_train: 0.5786 loss_val: 1.2217 acc_val: 0.7167 time: 4.7081s
Epoch: 0010 loss_train: 1.3469 acc_train: 0.5500 loss_val: 1.1692 acc_val: 0.7367 time: 4.6502s
Epoch: 0011 loss_train: 1.3209 acc_train

<All keys matched successfully>

In [33]:
# 理解索引的使用
a = torch.tensor([1, 2, 3, 4, 5])
b = range(1, 4)
c = torch.tensor(b)
d = torch.tensor([2, 4])
print(b)
print(c)
print(d)
print(a[b])
print(a[c])
print(a[d])

range(1, 4)
tensor([1, 2, 3])
tensor([2, 4])
tensor([2, 3, 4])
tensor([2, 3, 4])
tensor([3, 5])


In [48]:
# 理解加载模型的过程
# # print(model)
# # files = glob.glob('*.pkl')
# # print(files)
# checkpoint = torch.load("{}.pkl".format(best_epoch), map_location="cpu")
# # print(checkpoint)
# model.load_state_dict(checkpoint, strict=True)
# print(model)

GAT(
  (attention_0): GraphAttentionLayer(1433->8)
  (attention_1): GraphAttentionLayer(1433->8)
  (attention_2): GraphAttentionLayer(1433->8)
  (attention_3): GraphAttentionLayer(1433->8)
  (attention_4): GraphAttentionLayer(1433->8)
  (attention_5): GraphAttentionLayer(1433->8)
  (attention_6): GraphAttentionLayer(1433->8)
  (attention_7): GraphAttentionLayer(1433->8)
  (out_att): GraphAttentionLayer(64->7)
)
